# Analysis Notebooks

The registration of analysis notebooks involves the following entities and activity:
- `AnalysisNotebookTemplate` (Entity)
- `AnalysisNotebookEnvironment` (Entity)
- `AnalysisNotebookResult` (Entity)
- `AnalysisNotebookExecution` (Activity)

`AnalysisNotebookExecution` represents the activity of executing a notebook. It should be linked to:

- `AnalysisNotebookTemplate` (optional link), containing:
	- the notebook template as an asset (.ipynb),
	- requirements.txt as an asset, and
	- input specifications as a JSON attribute.
- `AnalysisNotebookEnvironment` (required link), containing:
    - frozen requirements.txt as an asset, and
    - runtime information as a JSON attribute.
- `AnalysisNotebookResult` (required generated output), containing:
    - the executed notebook as an asset (.ipynb).
- `Entities` (optional used input), linking one or more entities used in the notebook.


## Example

### Import modules

In [ ]:
import io
import os

from entitysdk import models
from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models.analysis_notebook_environment import (
    DockerRuntimeInfo,
    OsRuntimeInfo,
    PythonRuntimeInfo,
    RuntimeInfo,
)
from entitysdk.models.analysis_notebook_template import (
    AnalysisNotebookTemplateInputType,
    AnalysisNotebookTemplateSpecifications,
    DockerDependency,
    PythonDependency,
)
from entitysdk import models
from entitysdk.types import AnalysisScale, AssetLabel, ContentType, EntityType
from rich import print as rprint


### Initialize token and project environment

In [ ]:
entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")

### Initialize the Client object

In [ ]:
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)


### Create an AnalysisNotebookTemplate

This step is required only if the template doesn't exist yet.

In [ ]:
template = models.AnalysisNotebookTemplate(
    authorized_public=False,
    name="Skeletonize cell mesh and extract spines",
    description="In this notebook...",
    scale=AnalysisScale.cellular,
    specifications=AnalysisNotebookTemplateSpecifications(
        schema_version=1,
        python=PythonDependency(
            version=">=3.10,<3.13",
        ),
        docker=DockerDependency(
            image_repository="openbraininstitute/obi-notebook-image",
            image_tag=">=2025.09.24-2",
            image_digest="3406990b6e4c7192317b6fdc5680498744f6142f01f0287f4ee0420d8c74063c",
            docker_version=">=20.10",
        ),
        inputs=[
            AnalysisNotebookTemplateInputType(
                name="mesh_ids",
                entity_type=EntityType.em_cell_mesh,
                is_list=True,
                count_min=1,
                count_max=3,
            )
        ],
    ),
)
template = client.register_entity(template)
rprint(template)

# use an in-memory buffer to upload the asset,
# or use client.upload_file() alternatively
buffer = io.BytesIO(b"...")
asset_ipynb = client.upload_content(
    entity_id=template.id,
    entity_type=models.AnalysisNotebookTemplate,
    file_content=buffer,
    file_name="skeletonize_cell_mesh_and_extract_spines.ipynb",
    file_content_type=ContentType.application_x_ipynb_json,
    asset_label=AssetLabel.jupyter_notebook,
)
rprint(asset_ipynb)

# use an in-memory buffer to upload the asset
# or use client.upload_file() alternatively
buffer = io.BytesIO(b"rich==14.1.0")
asset_requirements = client.upload_content(
    entity_id=template.id,
    entity_type=models.AnalysisNotebookTemplate,
    file_content=buffer,
    file_name="requirements.txt",
    file_content_type=ContentType.text_plain,
    asset_label=AssetLabel.requirements,
)
rprint(asset_requirements)


### Register the runtime environment

In [ ]:
environment = models.AnalysisNotebookEnvironment(
    authorized_public=False,
    runtime_info=RuntimeInfo(
        schema_version=1,
        python=PythonRuntimeInfo(
            version="3.9.21",  # platform.python_version()
            implementation="CPython",  # platform.python_implementation()
            executable="/usr/bin/python",  # sys.executable
        ),
        docker=DockerRuntimeInfo(
            image_repository="openbraininstitute/obi-notebook-image",
            image_tag="2025.09.24-2",
            image_digest="3406990b6e4c7192317b6fdc5680498744f6142f01f0287f4ee0420d8c74063c",
            docker_version="28.4.0",
        ),
        os=OsRuntimeInfo(
            system="Linux",  # platform.system()
            release="5.14.0-427.28.1.el9_4.x86_64",  # platform.release()
            version="#1 SMP PREEMPT_DYNAMIC Fri Aug 2 03:44:10 EDT 2024",  # platform.version()
            machine="x86_64",  # platform.machine()
            processor="x86_64",  # platform.processor()
        ),
    ),
)
environment = client.register_entity(environment)

# use an in-memory buffer to upload the asset
# or use client.upload_file() alternatively
buffer = io.BytesIO(b"rich==14.1.0")
asset_requirements = client.upload_content(
    entity_id=environment.id,
    entity_type=models.AnalysisNotebookEnvironment,
    file_content=buffer,
    file_name="requirements.txt",
    file_content_type=ContentType.text_plain,
    asset_label=AssetLabel.requirements,
)
rprint(asset_requirements)


### Register the result

In [ ]:
result = models.AnalysisNotebookResult(
    authorized_public=False,
    name="Test notebook name",
    description="Test notebook result",
)
result = client.register_entity(result)
rprint(result)

# use an in-memory buffer to upload the asset,
# or use client.upload_file() alternatively
buffer = io.BytesIO(b"...")
asset_ipynb = client.upload_content(
    entity_id=result.id,
    entity_type=models.AnalysisNotebookResult,
    file_content=buffer,
    file_name="skeletonize_cell_mesh_and_extract_spines_result.ipynb",
    file_content_type=ContentType.application_x_ipynb_json,
    asset_label=AssetLabel.jupyter_notebook,
)
rprint(asset_ipynb)



### Register the activity

In [ ]:
execution = models.AnalysisNotebookExecution(
    authorized_public=False,
    start_time="2025-11-03T08:40:59.794317Z",
    end_time="2025-11-03T08:45:00.000000Z",
    used= [],  # add the list of input entities, if any
    generated= [result],
    analysis_notebook_template=template,
    analysis_notebook_environment=environment,
)
execution = client.register_entity(execution)
rprint(execution)

### Search and retrieval

The entities and activity can be searched and retrieved as usual.

In [ ]:
templates = client.search_entity(entity_type=models.AnalysisNotebookTemplate)
for t in templates:
    rprint(t)